In [1]:
import tensorflow as tf
import math
import json

/Users/ankit/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/ankit/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.enable_eager_execution()

In [3]:
class DNNRegressor(tf.estimator.Estimator):
    def __init__(self, feature_columns, outputs, hidden_units=2, **kwargs):
        super().__init__(model_fn=self._model_fn, params={
            'feature_columns': feature_columns,
            'hidden_units': hidden_units,
            'outputs': outputs
        },**kwargs)
    
    def _model_fn(self, features, labels, mode, params):
        inp = tf.feature_column.input_layer(features, params['feature_columns'])
        h = tf.layers.dense(inp, units=params['hidden_units'], activation=tf.nn.relu,name='hidden')
        out = tf.layers.dense(h, units=params['outputs'], activation=None,name="output")

        if mode == tf.estimator.ModeKeys.PREDICT:
            predictions = {
                'logits': out
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

        # Compute loss.
        loss = tf.losses.mean_squared_error(labels=labels,predictions=out)
        # Compute evaluation metrics.
        precision = tf.metrics.precision(labels=labels,
                                       predictions=out,
                                       name='pre_op')
        recall = tf.metrics.recall(labels=labels,
                                       predictions=out,
                                       name='rec_op')
        metrics = {}
        #metrics = {'precision': precision,'recall':recall}
        #tf.summary.scalar('precision', precision[1])
        #tf.summary.scalar('recall', recall[1])

        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(
                mode, loss=loss, eval_metric_ops=metrics)

        # Create training op.
        assert mode == tf.estimator.ModeKeys.TRAIN

        optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

    def get_layer_params(self, layer_name):
        return self.get_variable_value('{}/kernel'.format(layer_name)), self.get_variable_value('{}/bias'.format(layer_name))

In [4]:
def input_fn_y1():
    d = tf.contrib.data.CsvDataset('algebra.csv',[tf.float32]*4)
    d = d.batch(32).map(lambda x1,x2,y1,y2:({"x1":x1,"x2":x2},tf.transpose(tf.stack([y1]))))
    return d

def input_fn_y2():
    d = tf.contrib.data.CsvDataset('algebra.csv',[tf.float32]*4)
    d = d.batch(32).map(lambda x1,x2,y1,y2:({"x1":x1,"x2":x2},tf.transpose(tf.stack([y2]))))
    return d

def input_fn_both():
    d = tf.contrib.data.CsvDataset('algebra.csv',[tf.float32]*4)
    d = d.batch(32).map(lambda x1,x2,y1,y2:({"x1":x1,"x2":x2},tf.transpose(tf.stack([y1,y2]))))
    return d

def train_data(input_fn):
    count = 0
    for i in input_fn():
        count += 1
    return lambda: input_fn().take(math.floor(count*0.85))
    
def test_data(input_fn):
    count = 0
    for i in input_fn():
        count += 1
    return lambda: input_fn().skip(math.ceil(count*0.85))


In [7]:
def train_models():
    d_y1.train(train_data(input_fn_y1))
    d_y2.train(train_data(input_fn_y2))
    d_both.train(train_data(input_fn_both))
    d_both_large.train(train_data(input_fn_both))

def eval_models():
    print("y1:")
    print(d_y1.evaluate(input_fn=test_data(input_fn_y1)))
    print("y2:")
    print(d_y2.evaluate(input_fn=test_data(input_fn_y2)))
    print("Both:")
    print(d_both.evaluate(input_fn=test_data(input_zfn_both)))
    print("Both(Large):")
    print(d_both_large.evaluate(input_fn=test_data(input_fn_both)))

    

In [8]:
feat_cols = [tf.feature_column.numeric_column('x1'), tf.feature_column.numeric_column('x2')]
d_y1 = DNNRegressor(feat_cols,outputs=1)
d_y2 = DNNRegressor(feat_cols,outputs=1)
d_both = DNNRegressor(feat_cols,outputs=2)
d_both_large = DNNRegressor(feat_cols,hidden_units=100,outputs=2)

train_models()
tf.logging.set_verbosity(tf.logging.WARN)    
eval_models()
tf.logging.set_verbosity(tf.logging.INFO)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_train_distribute': None, '_service': None, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_device_fn': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c24c13c50>, '_model_dir': '/var/folders/60/h5y4q3zs4fl_wkw_007jnhp80000gn/T/tmpxje888sf', '_num_ps_replicas': 0, '_task_type': 'worker', '_tf_random_seed': None, '_master': '', '_is_chief': True, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_log_step_count_steps': 100}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_train_distribute': None, '_service': None, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_de

NameError: name 'input_zfn_both' is not defined

In [47]:
(w1,w2), (b1,b2) = d_y1.get_layer_params('hidden') 
(w0),(b0,) = d_y1.get_layer_params('output')
w0 = w0.T[0]